In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI TensorBoard预构建容器进行自定义训练

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_custom_training_with_prebuilt_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tensorboard/tensorboard_custom_training_with_prebuilt_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tensorboard/tensorboard_custom_training_with_prebuilt_container.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

### 什么是Vertex AI TensorBoard

Vertex AI TensorBoard是一个经过企业级管理的版本，是[开源TensorBoard](https://www.tensorflow.org/tensorboard/get_started) (TB)的一个Google开源项目，用于机器学习实验可视化。

Vertex AI TensorBoard提供了各种详细的可视化功能，包括：

* 跟踪和可视化指标，如损失和准确率随时间变化，
* 可视化模型计算图（操作和层），
* 查看权重、偏置或其他张量随时间变化的直方图，
* 投影嵌入到较低维度空间，
* 显示图像、文本和音频样本。

除了来自TensorBoard的强大可视化功能外，Vertex AI TensorBoard还提供：

* 实验仪表板的持久、可共享链接，
* 项目中所有实验的可搜索列表，
* 与Vertex AI服务紧密集成，用于模型训练，
* 企业级的安全性、隐私性和合规性。

使用Vertex AI TensorBoard，您可以跟踪、可视化和比较机器学习实验，并与团队共享。

了解更多关于[Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction)和[自定义训练](https://cloud.google.com/vertex-ai/docs/training/overview)的信息。

### 目标

在本教程中，您将学习如何使用预先构建的容器创建自定义训练作业，并在 Vertex AI TensorBoard 上实时监控您的训练过程。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 训练
- Vertex AI TensorBoard

执行的步骤包括：

* 设置服务账号和 Google Cloud Storage 存储桶。
* 编写您的定制训练代码。
* 打包并上传您的训练代码至 Google Cloud Storage。
* 创建并启动启用 Tensorboard 的自定义训练作业，以进行实时监控。

数据集

本教程中使用的数据集是由TensorFlow提供的[花卉数据集]。不需要其他数据集。

### 成本

本教程使用 Google Cloud 的收费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用情况生成成本估算。

### 安装额外的软件包

安装以下软件包以执行此笔记本所需的功能。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform 

###仅限Colab：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager).

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [启用以下 API：Vertex AI API, Cloud Resource Manager API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com).

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk).

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 地区

您也可以更改 Vertex AI 使用的“REGION”变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动验证。请按照下面的相关说明进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经通过验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务账户授予Cloud Storage权限。

### 创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间产物。

当您使用Cloud SDK提交一个训练作业时，您会将包含训练代码的Python软件包上传到一个云存储桶中。Vertex AI会从这个软件包中运行代码。在本教程中，Vertex AI还会将您作业产生的训练模型保存在同一个存储桶中。通过使用这个模型产物，您可以创建一个Vertex AI模型资源，并用于预测。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

设置服务账户和权限##

一个服务账号用来创建自定义的训练作业。如果您不想使用您项目的Compute Engine服务账号，请将SERVICE_ACCOUNT设置为另一个服务账号ID。您可以通过按照 [说明](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating) 创建一个服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = ! gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
# Grant Cloud Storage permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
   --member=serviceAccount:$SERVICE_ACCOUNT \
   --role=roles/storage.admin \
   --quiet

In [ ]:
# Grant AI Platform permission.
! gcloud projects add-iam-policy-binding $PROJECT_ID \
   --member=serviceAccount:$SERVICE_ACCOUNT \
   --role=roles/aiplatform.user \
   --quiet

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform

### 为 Python 初始化 Vertex AI SDK
为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## 编写您的训练代码
您的训练代码必须配置为将TensorBoard日志编写到Cloud Storage存储桶中，Vertex AI Training服务将通过预定义的环境变量 `AIP_TENSORBOARD_LOG_DIR` 自动提供其位置。

通常可以通过将 `os.environ['AIP_TENSORBOARD_LOG_DIR']` 提供为开源TensorBoard日志编写API的日志目录来实现这一点。

例如，在TensorFlow 2.x中，您可以使用以下代码创建一个 `tensorboard_callback`：
```
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)
```

`AIP_TENSORBOARD_LOG_DIR` 将位于您在创建自定义训练作业时提供的 `BASE_OUTPUT_DIR` 中。

我们将使用以下示例代码作为示例：

In [ ]:
# Download the sample code
! gsutil cp gs://cloud-samples-data/ai-platform/hello-custom/hello-custom-sample-v1.tar.gz - | tar -xzv
%cd hello-custom-sample/

In [ ]:
# The training code we want to edit is:
! cat trainer/task.py

在 `trainer/task.py` 中创建 `tensorboard_callback`，然后将该回调添加到 `model.fit(...)`。

示例代码：
```
# 其他代码
model.compile(...)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)
  
model.fit(dataset, epochs=10, callbacks=[tensorboard_callback])
```

在 `trainer/task.py` 中更新您的代码，加入 `tensorboard_callback`。您可以使用上面的示例代码。

In [ ]:
%%writefile trainer/task.py

import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds

IMG_WIDTH = 128


def normalize_img(image):
    """Normalizes image.

    * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
    * Casts values from `uint8` to `float32`
    * Scales values from [0, 255] to [0, 1]

    Returns:
      A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
    """
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    return image / 255.


def normalize_img_and_label(image, label):
    """Normalizes image and label.

    * Performs normalize_img on image
    * Passes through label unchanged

    Returns:
      Tuple (image, label) where
      * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
        channels)
      * label is an unchanged integer [0, 4] representing flower type
    """
    return normalize_img(image), label


if 'AIP_MODEL_DIR' not in os.environ:
    raise KeyError(
        'The `AIP_MODEL_DIR` environment variable has not been' +
        'set. See https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom/training'
    )
output_directory = os.environ['AIP_MODEL_DIR']

logging.info('Loading and preprocessing data ...')
dataset = tfds.load('tf_flowers:3.*.*',
                    split='train',
                    try_gcs=True,
                    shuffle_files=True,
                    as_supervised=True)
dataset = dataset.map(normalize_img_and_label,
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(128)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

logging.info('Creating and training model ...')
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,
                           3,
                           padding='same',
                           activation='relu',
                           input_shape=(IMG_WIDTH, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(5)  # 5 classes
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

### Create a TensorBoard call back and write to the gcs path provided by AIP_TENSORBOARD_LOG_DIR
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'],
  histogram_freq=1)

### Train the model with tensorboard_callback
model.fit(dataset, epochs=14, callbacks=[tensorboard_callback])

logging.info(f'Exporting SavedModel to: {output_directory}')
# Add softmax layer for intepretability
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
probability_model.save(output_directory)

## 将您的训练代码上传到云存储

为了让Vertex AI在自定义训练流程中运行代码，您必须将训练代码打包为源分发，并将其上传到云存储。

1. 运行以下命令以创建一个gzipped tarball格式的源分发。该命令使用示例代码中包含的`setup.py`文件。

In [ ]:
! python3 setup.py sdist --formats=gztar

运行以下命令将刚刚创建的源分发文件`dist/hello-custom-training-3.0.tar.gz`上传到您之前创建的云存储存储桶。

In [ ]:
GCS_BUCKET_TRAINING = f"{BUCKET_URI}/data/"
! gsutil cp dist/hello-custom-training-3.0.tar.gz {GCS_BUCKET_TRAINING}

创建一个自定义的训练任务

创建一个TensorBoard实例，供自定义训练任务使用。

In [ ]:
TENSORBOARD_NAME = "[your-tensorboard-name]"  # @param {type:"string"}

if (
    TENSORBOARD_NAME == ""
    or TENSORBOARD_NAME is None
    or TENSORBOARD_NAME == "[your-tensorboard-name]"
):
    TENSORBOARD_NAME = PROJECT_ID + "-tb"

tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=REGION
)
TENSORBOARD_RESOURCE_NAME = tensorboard.gca_resource.name
print("TensorBoard resource name:", TENSORBOARD_RESOURCE_NAME)

运行以下示例请求，创建您自己的自定义训练作业，并将训练结果流式传输到TensorBoard。

In [ ]:
JOB_NAME = "tensorboard-example-job"
GCS_BUCKET_OUTPUT = BUCKET_URI
BASE_OUTPUT_DIR = "{}/{}".format(GCS_BUCKET_OUTPUT, JOB_NAME)

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=JOB_NAME,
    python_module_name="trainer.task",
    container_uri="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest",
    python_package_gcs_uri=f"{GCS_BUCKET_TRAINING}hello-custom-training-3.0.tar.gz",
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BASE_OUTPUT_DIR,
)

job.run(
    service_account=SERVICE_ACCOUNT,
    tensorboard=TENSORBOARD_RESOURCE_NAME,
    machine_type="n1-standard-8",
    replica_count=1,
)

在Google Cloud控制台中，您可以在Vertex AI > 训练 > 自定义作业中监视您的训练作业。在每个自定义训练作业中，可以通过“打开TensorBoard”按钮查看近实时更新的TensorBoard。了解更多信息，请参阅[查看Vertex AI TensorBoard数据]（https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-view）。

## 清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shuttind_down_projects)。

否则，**如果您在笔记本中创建了个别资源**，您可以按以下步骤删除它们：

In [ ]:
# Delete GCS bucket.
! gsutil -m rm -r {BUCKET_URI}

# Delete TensorBoard instance.
! gcloud ai tensorboards delete {TENSORBOARD_RESOURCE_NAME}

# Delete custom job.
job.delete()